In [44]:
import torch
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import sys
# sys.path.append("..")
import utils.utils as utils
import utils.adv_ex_utils as aus
import ir_utils.wide_resnet as wide_resnet
import ir_utils.dataloaders
from advertorch.attacks import GradientSignAttack, CarliniWagnerL2Attack, PGDAttack
import statistics as stats
import scipy.stats

# makes default tensor a CUDA tensor so GPU can be used
device = torch.device(1 if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
if device != 'cpu':
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [45]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

In [46]:
def test(net):
    correct = 0
    adv_correct = 0
    
    for samples, labels in test_loader:
        samples, labels = samples.to(device), labels.to(device)
                
        # for perturbed samples
        adv_samples, adv_labels = aus.generate_adv_exs(samples, labels, adversary, num_per_samp=1)
        adv_output = net(adv_samples)
        adv_preds = adv_output.data.max(1, keepdim=True)[1]
        adv_correct += adv_preds.eq(adv_labels.data.view_as(adv_preds)).sum().item()
        
        # for unperturbed test samples
        output = net(samples)
        preds = output.data.max(1, keepdim=True)[1]
        correct += preds.eq(labels.data.view_as(preds)).sum().item()

    adv_test_accuracy = 100. * float(adv_correct / len(test_loader.dataset))
    test_accuracy = 100. * float(correct / len(test_loader.dataset))
    
    test_accuracies.append(test_accuracy)
    adv_test_accuracies.append(adv_test_accuracy)

In [48]:
d = {}

MNIST_path = '../trained_models/MNIST/simplecnn_{}/model{}'
MNIST_model_types = ['stdtrain', 'at', 'jr', 'ir', 'ir_permuted/rd1', 'ir_st_smoothgrad'] #'at', , ]

CIFAR_10_path = '../trained_models/CIFAR-10/resnet_{}/model{}'
CIFAR_10_model_types = ['stdtrain']#,'at','jr','ir']

for model_type in CIFAR_10_model_types:
    adv_test_accuracies = []
    test_accuracies = []
    
    for i in range(6,11):
        net = resnet.resnet20()
        adversary = PGDAttack(predict=net, loss_fn=F.cross_entropy, eps=8/255., 
                      nb_iter=20, eps_iter=2/255., rand_init=True,
                      clip_min=0., clip_max=1., ord=np.inf, targeted=False)
#         adversary = CarliniWagnerL2Attack(predict=net, num_classes=10, confidence=0,
#                  targeted=False, learning_rate=0.01,
#                  binary_search_steps=9, max_iterations=10000,
#                  abort_early=True, initial_const=1e-3,
#                  clip_min=0., clip_max=1., loss_fn=None)
        try:
            net.load_state_dict(torch.load(CIFAR_10_path.format(model_type, i)))
        except:
            print('CANNOT LOAD MODEL: ', model_type, i)
            continue
        net.eval()
        test(net)
            
    adv_ci = mean_confidence_interval(adv_test_accuracies)
    std_ci = mean_confidence_interval(test_accuracies)
    
    d[model_type] = [adv_ci, std_ci]

In [51]:
for m in d:
    print(f'{m}: {d[m][1][0]:.2f} +/- {d[m][1][1]:.2f}')

stdtrain: 91.91 +/- 0.38
